<a href="https://colab.research.google.com/github/QuentinNativel/ELTA/blob/master/ELTA_project_combined.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import re
import spacy
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm
from sklearn import tree
from sklearn.ensemble import BaggingClassifier,RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Data loading

In [0]:
x_path = "/content/drive/My Drive/ELTA/X_train_update.csv"
y_path = "/content/drive/My Drive/ELTA/Y_train_CVw08PX.csv"
x_df = pd.read_csv(x_path)
y_df = pd.read_csv(y_path)
# drop useless columns
x_df.drop(columns=['Unnamed: 0', 'description', 'productid', 'imageid'], axis=0, inplace=True)

### Tokenization
At first we clean the accent using the function normalize accent

In [0]:
def normalize_accent(string):
    string = string.replace('á', 'a')
    string = string.replace('â', 'a')

    string = string.replace('é', 'e')
    string = string.replace('è', 'e')
    string = string.replace('ê', 'e')
    string = string.replace('ë', 'e')

    string = string.replace('î', 'i')
    string = string.replace('ï', 'i')

    string = string.replace('ö', 'o')
    string = string.replace('ô', 'o')
    string = string.replace('ò', 'o')
    string = string.replace('ó', 'o')

    string = string.replace('ù', 'u')
    string = string.replace('û', 'u')
    string = string.replace('ü', 'u')

    string = string.replace('ç', 'c')
    string = re.sub(r'°|#|/|-|%|_|[0-9]+', '', string)
    
    return string

In [28]:
normalize_accent('journal arts n° 133 28/09/2001 art marche salo...')

'journal arts n   art marche salo...'

In [29]:
!python -m spacy download fr


✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/fr_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/fr
You can now load the model via spacy.load('fr')


In [0]:
# Load spaCy for french
spacy_nlp = spacy.load("fr")

In [0]:
def raw_to_tokens(raw_string, spacy_nlp):
    # Write code for lower-casing
    string = raw_string.lower().rstrip()
    
    # Write code to normalize the accents
    string = normalize_accent(string)
        
    # Write code to tokenize
    spacy_tokens = spacy_nlp(string)
        
    # Write code to remove punctuation tokens and create string tokens
    string_tokens = [token.orth_ for token in spacy_tokens if not token.is_punct if not token.is_stop]
    
    # Write code to join the tokens back into a single string
    clean_string = " ".join(string_tokens)
    
    return clean_string

In [32]:
x_df['tokens'] = x_df['designation'].apply(lambda s : raw_to_tokens(s,spacy_nlp))
x_df.to_csv('./tokens.csv')

KeyboardInterrupt: ignored

In [5]:
x_df = pd.read_csv('/content/drive/My Drive/ELTA/tokens.csv')
x_df.tail()

,Unnamed: 0,designation,tokens
84911,84911,The Sims [ Import Anglais ],the sims import anglais
84912,84912,Kit piscine acier NEVADA déco pierre Ø 3.50m x...,kit piscine acier nevada deco pierre ø .m x .m
84913,84913,Journal Officiel De La Republique Francaise N°...,journal officiel republique francaise n c...
84914,84914,Table Basse Bois De Récupération Massif Base B...,table basse bois recuperation massif base blan...
84915,84915,Gomme De Collection 2 Gommes Pinguin Glace Ver...,gomme collection gommes pinguin glace vert o...


In [40]:
y_df.prdtypecode.value_counts()

2583    10209
1560     5073
1300     5045
2060     4993
2522     4989
1280     4870
2403     4774
2280     4760
1920     4303
1160     3953
1320     3241
10       3116
2705     2761
1140     2671
2582     2589
40       2508
2585     2496
1302     2491
1281     2070
50       1681
2462     1421
2905      872
60        832
2220      824
1301      807
1940      803
1180      764
Name: prdtypecode, dtype: int64

In [6]:
# drop na
x_df, y_df = x_df[ ~ x_df.tokens.isna()], y_df[~ x_df.tokens.isna()]
print(x_df.shape, y_df.shape)

(84905, 3) (84905, 2)


In [0]:
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(x_df['tokens'])

In [0]:
y = y_df.prdtypecode

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [0]:
weighted_f1_scores = {}

In [0]:
#Decision tree

In [49]:
clf = tree.DecisionTreeClassifier()
clf.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [0]:
y_pred = clf.predict(X_test)
weighted_f1_scores["decision tree"] = f1_score(y_test, y_pred,average="weighted")

In [0]:
# Bagging

In [0]:
clf_bag = BaggingClassifier(tree.DecisionTreeClassifier(), max_samples=0.5)
clf_bag.fit(X_train, y_train)

In [0]:
y_pred_bag = clf_bag.predict(X_test)
weighted_f1_scores["bagging"]  = f1_score(y_test, y_pred_bag,average="weighted")

In [0]:
# Random forest

In [0]:
clf_rf = RandomForestClassifier(n_estimators = 100,max_depth = 100)
clf_rf.fit(X_train,y_train)

In [0]:
y_pred_rf = clf_rf.predict(X_test)
weighted_f1_scores["random forest classifier"]  = f1_score(y_test, y_pred_rf,average="weighted")

In [0]:
weighted_f1_scores

{'random forest classifier': 0.6487346355423826}

In [0]:
#adaboost

In [0]:
estimators = 70
alpha = 0.01
random = None
model = AdaBoostClassifier(n_estimators = estimators, learning_rate = alpha, random_state = random )

In [0]:
def evaluate_hyperparameters(estimators,alpha,random,X_train,y_train,X_test,y_test):
    model = AdaBoostClassifier(n_estimators = estimators, learning_rate = alpha, random_state = random )
    res = model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return f1_score(y_test, y_pred,average="weighted")

In [0]:
results_adaboost = {elt:[] for elt in ["estimators","alpha","random","weighted_f1_score"] }

In [0]:
n_estimators = [50,100]
alpha = [0.01,0.05,0.1,0.3,1]
estimators = n_estimators[0]
random = None
for alpha in tqdm(alpha):
    results["estimators"] = estimators
    results["alpha"] = alpha
    results["random"] = random
    results["weighted_f1_score"] = evaluate_hyperparameters(estimators,alpha,random,X_train,y_train,X_test,y_test)

  0%|          | 0/5 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [0]:
res = model.fit(X_train, y_train)

KeyboardInterrupt: 

In [0]:
model.score(X_train, y_train)

In [0]:
y_pred = model.predict(X_test)
f1_score(y_test, y_pred,average="weighted")

TypeError: only integer scalar arrays can be converted to a scalar index

In [0]:
param_dist = {
 'n_estimators': [50, 100],
 'learning_rate' : [0.1,0.3,1] }
pre_gs_inst = RandomizedSearchCV(AdaBoostClassifier(),
 param_distributions = param_dist,
 cv=3,
 n_iter = 5,
 n_jobs=-1)

In [0]:
pre_gs_inst.fit(X_train, y_train)